# Homework Batch III: Routing Algorithms
## Davide Basso - SM3500450

1. Implement the binary heap-based version of the Dijkstra's algorithm.

In [1]:
from math import inf

In [2]:
class Node:

    def __init__(self, index, heap_idx, dist, pred):
        self.index = index
        self.heap_index = heap_idx
        self.d = dist
        self.pred = pred
    
    def print_node(self):
        print(f"index: {self.index}, data: {self.d}, predecessor: {self.pred}")

class Edge:
    
    def __init__(self, src, dest, weight=1):
        self.u = src
        self.v = dest
        self.weight = weight

class Weighted_Graph:

    def __init__(self, vertices, edges):
        self.V = [vertix for vertix in vertices]
        self.Adj = [[] for vertix in range(len(vertices))]

        for edge in edges:
            self.Adj[edge.u.index].append((edge.v,edge.weight))
    
    def print_adj(self):
        for el in G.V:
            print(el.index)
            for j in el:
                print(f"\t ->{j[0].index}")

In [3]:
from typing import TypeVar, Generic, Union, List

from numbers import Number

T = TypeVar('T')

def min_order(a: Number, b: Number) -> bool:
    return a <= b

def max_order(a: Number, b: Number) -> bool:
    return a >= b

class binheap(Generic[T]):

    def __init__(self, A: Union[int, List[T]], total_order = None):

        if total_order is None:
            self._torder = min_order
        else:
            self._torder = max_order

        if isinstance(A, int):
            self._size = 0
            self._A = [None]*A
        else:
            self._size = len(A)
            self._A = A
        
        self._build_heap()

    @staticmethod
    def parent(node: int) -> Union[int, None]:
        if node == 0:
            return None

        return (node-1)//2

    @staticmethod
    def child(node: int, side: int) -> int:
        return 2*node + 1 + side

    @staticmethod
    def left(node: int) -> int:
        return 2*node + 1

    @staticmethod
    def right(node: int) -> int:
        return 2*node + 2
    
    def is_empty(self) -> bool:
        return self._size == 0
 
    def __len__(self):
        return self._size

    def _swap_keys(self, node_a: int, node_b: int) -> None:
        tmp = self._A[node_a]
        self._A[node_a] = self._A[node_b]
        self._A[node_b] = tmp

        tmp = self._A[node_a].heap_index
        self._A[node_a].heap_index = self._A[node_b].heap_index
        self._A[node_b].heap_index = tmp

    # iterative version
    def _heapify(self, node: int) -> None:
        keep_fixing = True

        while keep_fixing:
            min_node = node
            for child_idx in [2*node + 1, 2*node + 2]:
                if child_idx < self._size and self._torder(self._A[child_idx].d, self._A[min_node].d):
                    min_node = child_idx

            # min_node is the index of the minimum key 
            # among the keys of root and its children

            if min_node != node:
                self._swap_keys(min_node, node)
                node = min_node
            else:
                keep_fixing = False

    def remove_minimum(self) -> T:
        if self.is_empty():
            raise RuntimeError('The heap is empty')

        self._swap_keys(0, self._size-1)

        self._size = self._size-1

        self._heapify(0)

        return self._A[self._size]

    def _build_heap(self) -> None:
        for i in range(binheap.parent(self._size-1), -1, -1):
            self._heapify(i)
    
    def decrease_key(self, node: int, new_value: T) -> None:
        self._A[node].d = new_value

        parent = binheap.parent(node)

        while node != 0 and not self._torder(self._A[parent].d, self._A[node].d):
            self._swap_keys(node, parent)
            node = parent
            parent = binheap.parent(node)

    def insert(self, value: T) -> None:
        if self._size >= len(self._A):
            raise RuntimeError('The heap is full')
            
        if self.is_empty():
            self._A[0] = value
            self._size += 1
        else:
            parent = binheap.parent(self._size)
            if self._torder(self._A[parent], value):
                self._A[self._size] = value
            else:
                self._A[self._size] = self._A[parent]
                
            self._size += 1
            self.decrease_key(self._size - 1, value)

    # nice print 
    def __repr__(self) ->str:
        br_str = ''

        # pseudocode indexes
        next_node = 1
        up_to = 2

        while next_node <= self._size:
            level = '\t'.join(f'{v.index},{v.d},{v.pred},{v.heap_index}' for v in self._A[next_node-1: min(up_to-1, self._size)])

            if next_node == 1:
                bh_str = level
            else:
                bh_str += f'\n{level}'

            next_node = up_to
            up_to = 2*up_to

        return bh_str


In [4]:
def init_SSSP(G):
    for v in G.V:
        v.d = inf
        v.pred = None

def relax(Q, u, v, w):
    #print(v.index, v.heap_index)
    if u.d+w < v.d:
        Q.decrease_key(v.heap_index, u.d+w)
        v.pred = u

def Dijkstra(G, s):
    init_SSSP(G)
    s.d = 0

    Q = binheap(G.V)
    for h_i, node in enumerate(Q._A):
        node.heap_index = h_i

    final_queue = []
    i = 0
    while not Q.is_empty():
        #print(Q)
        u = Q.remove_minimum()
        final_queue.append(u)

        # this means that we've arrived at the point of the queue
        # in which are located only nodes eliminated by shortcuts
        if u.d == inf and u.pred is None:
            break

        if i != 0:
            print(f"\nnode {u.index} with total distance {u.d} from root and predecessor node is {u.pred.index}")
        else:
            i += 1
            print(f"The path starts from node {u.index}, then we have:")

        for (v,w) in G.Adj[u.index]:
            relax(Q, u, v, w)
    
    return final_queue

In [5]:
a = Node(0,None,None,None)
b = Node(1,None,None,None)
c = Node(2,None,None,None)
d = Node(3,None,None,None)
e = Node(4,None,None,None)
f = Node(5,None,None,None)

vertices = [a,b,c,d,e,f]
edges = [Edge(a,b,1), Edge(a,c,5), Edge(b,f,15), Edge(c,d,2), Edge(d,e,1), Edge(e,f,3)]

In [7]:
G = Weighted_Graph(vertices, edges)
for v in G.V:
    v.print_node()

index: 0, data: None, predecessor: None
index: 1, data: None, predecessor: None
index: 2, data: None, predecessor: None
index: 3, data: None, predecessor: None
index: 4, data: None, predecessor: None
index: 5, data: None, predecessor: None


In [18]:
res = Dijkstra(G,a)

The path starts from node 0, then we have:

node 1 with total distance 1 from root and predecessor node is 0

node 2 with total distance 5 from root and predecessor node is 0

node 3 with total distance 7 from root and predecessor node is 2

node 4 with total distance 8 from root and predecessor node is 3

node 5 with total distance 11 from root and predecessor node is 4


2. Consider the contraction hierarchies presented during the course. Assume to deal with graphs that can be fully represented in the memory of your computer. Implement:

(a) an algorithm to add the shortcuts to a graph;


In [9]:
def compute_shortcut(node, edges, vertices):
    shortcuts = {"deleted": None,
                 "pred": [], "pred_weight":[],
                 "next": [], "next_weight": []}

    shortcuts["deleted"] = node.index

    # find the edges in which node is involved
    # and add them to the dictionary
    for edge in edges:
        if edge.v.index == node.index:
            shortcuts["pred"].append(edge.u.index)
            shortcuts["pred_weight"].append(edge.weight)
        if edge.u.index == node.index:
            shortcuts["next"].append(edge.v.index)
            shortcuts["next_weight"].append(edge.weight)

    for k,v in shortcuts.items():
        print(f"{k} -> {v}")
    
    # remove edges involving node
    edges = [edge for edge in edges if not (edge.v.index == node.index or edge.u.index == node.index)]
    
    # create new edges
    new_edges = []
    for pred, pred_w in zip(shortcuts["pred"], shortcuts["pred_weight"]):
        #print(pred,"and", pred_w)
        p = [v for v in vertices if v.index == pred][0]
        for succ, succ_w in zip(shortcuts["next"], shortcuts["next_weight"]):
            #print(succ,"and", succ_w)
            s = [v for v in vertices if v.index == succ][0]
            new_edge = Edge(p, s, pred_w+succ_w)
            new_edges.append(new_edge)
            edges.append(new_edge)
    
    # compare if new edges are worse than already existing ones
    # if so remove them
    for n_e in new_edges:
        for e in edges:
            if e.u.index == n_e.u.index and e.v.index == n_e.v.index and e.weight < n_e.weight:
                edges.remove(n_e)
                
    return edges

In [57]:
a = Node(0,None,None,None)
b = Node(1,None,None,None)
c = Node(2,None,None,None)
d = Node(3,None,None,None)
e = Node(4,None,None,None)
f = Node(5,None,None,None)

vertices = [a,b,c,d,e,f]
edges = [Edge(a,b,10), Edge(a,c,1), Edge(b,c,3), Edge(b,f,15), Edge(c,d,2), Edge(d,e,1), Edge(e,f,3)]

In [11]:
res = compute_shortcut(c, edges, vertices)

print("\n")
for el in res:
    print(f"pred: {el.u.index}, succ: {el.v.index}, val: {el.weight}")

deleted -> 2
pred -> [0, 1]
pred_weight -> [1, 3]
next -> [3]
next_weight -> [2]


pred: 0, succ: 1, val: 10
pred: 1, succ: 5, val: 15
pred: 3, succ: 4, val: 1
pred: 4, succ: 5, val: 3
pred: 0, succ: 3, val: 3
pred: 1, succ: 3, val: 5


In [12]:
G = Weighted_Graph(vertices, res)
res = Dijkstra(G,a)

The path starts from node 0, then we have:

node 3 with total distance 3 from root and predecessor node is 0

node 4 with total distance 4 from root and predecessor node is 3

node 5 with total distance 7 from root and predecessor node is 4

node 1 with total distance 10 from root and predecessor node is 0


(b) a bidirectional version of Dijkstra algorithm that can operate on the graphs decorated by the algorithm at Point 2a.

In [5]:
def reverse_edges(edges):
    back = []
    for el in edges:

        tmp = Edge(el.v, el.u, el.weight)
        back.append(tmp)

    return back

In [8]:
def get_predecessors(node, predecs):
    if node.pred is not None:
        predecs.append(node.pred)
        get_predecessors(node.pred, predecs)

In [74]:
from copy import deepcopy
def bidirect_Dijkstra(f_G, b_G, s, t):
    F_G = deepcopy(f_G)
    B_G = deepcopy(b_G)
    init_SSSP(F_G)
    init_SSSP(B_G)

    F_G.V[s.index].d = 0
    f_Q = binheap(F_G.V)

    B_G.V[t.index].d = 0
    b_Q = binheap(B_G.V)

    for h_i, node in enumerate(f_Q._A):
        node.heap_index = h_i
    for h_i, node in enumerate(b_Q._A):
        node.heap_index = h_i
    
    #print(f_Q)
    #print(b_Q)

    forward_processed = []
    backward_processed = []
    i = 0
    final_weight = 0
    while not f_Q.is_empty() and not b_Q.is_empty():
        #print(f_Q)
        #print(b_Q)
        u = f_Q.remove_minimum()
        v = b_Q.remove_minimum()

        flag = False

        f_queue = []
        b_queue = []
        for el in backward_processed:
            if u.index == el.index:
                flag = True
                get_predecessors(u,f_queue)
        
        for el in forward_processed:
            if v.index == el.index:
                flag = True
                get_predecessors(v,b_queue)


        if flag is True:
            f_queue.reverse()
            final_weight = f_queue[-1].d + b_queue[0].d
            f_queue.pop()
            break
        print(u.index,v.index)

        forward_processed.append(u)
        backward_processed.append(v)
        
        
        # this means that we've arrived at the point of the queue
        # in which are located only nodes eliminated by shortcuts
        if u.d == inf and u.pred is None or v.d == inf and v.pred is None:
            break
        
        if i != 0:
            print(f"\nforward node {u.index} with total distance {u.d} from root and predecessor node is {u.pred.index}")
            print(f"backward node {v.index} with total distance {v.d} from root and predecessor node is {v.pred.index}")
        else:
            i += 1
            print(f"The forward path starts from node {u.index}, then we have:")
            print(f"The backward path starts from node {v.index}, then we have:")
        

        for (k,w) in F_G.Adj[u.index]:
            relax(f_Q, u, k, w)
            #if k in backward_processed and u.d + k.d + w < mu:
            #    mu = u.d + k.d + w  
        
        for (k,w) in B_G.Adj[v.index]:
            relax(b_Q, v, k, w)
            #if k in forward_processed and v.d + k.d + w < mu:
            #    mu = v.d + k.d + w

    return f_queue+b_queue, final_weight


In [75]:
a = Node(0,None,None,None)
b = Node(1,None,None,None)
c = Node(2,None,None,None)
d = Node(3,None,None,None)
e = Node(4,None,None,None)
f = Node(5,None,None,None)

vertices = [a,b,c,d,e,f]
edges = [Edge(a,b,1), Edge(a,c,10), Edge(b,c,1), Edge(b,f,15), Edge(c,d,2), Edge(d,e,1), Edge(e,f,3)]
backward_edges = reverse_edges(edges)

G1 = Weighted_Graph(vertices, edges)
G2 = Weighted_Graph(vertices, backward_edges)

In [76]:
for el in edges:
    print(el.u.index, el.v.index)
print("\n")
for el in backward_edges:
    print(el.u.index, el.v.index)

0 1
0 2
1 2
1 5
2 3
3 4
4 5


1 0
2 0
2 1
5 1
3 2
4 3
5 4


In [77]:
res, weight = bidirect_Dijkstra(G1, G2, a, e)

0 4
The forward path starts from node 0, then we have:
The backward path starts from node 4, then we have:
1 3

forward node 1 with total distance 1 from root and predecessor node is 0
backward node 3 with total distance 1 from root and predecessor node is 4
2 2

forward node 2 with total distance 2 from root and predecessor node is 1
backward node 2 with total distance 3 from root and predecessor node is 3
